In [1]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from loadTDMS import *
from fft_test import *
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
# Normalization class
mmsc=MinMaxScaler()
# Standardize class
stdsc = StandardScaler()

In [3]:
def make_trainD(case, tool, start, end, fq, AE_s_l):
    
    if tool == "A":
        date = "20230113+0117"
        interval = 1200
    if tool == "B":
        date = "20230120"
        interval = 1200
    if tool == "C":
        date = "20230123"
        interval = 1023
    
    d = np.array([])

    for i in range(start,end+1):
        d_n = np.load(rf"Z:\200 Produced_data\{date}\FFT\{AE_s_l}\{i}.npy")
        d_n = d_n.reshape(interval,820)
        d_n = d_n.reshape(interval,10,82)
        d_n = d_n[:,[fq]]
        d_n = d_n.reshape(interval,82*len(fq))
        d_n = d_n.ravel()
        d = np.append(d,d_n)
    
    d_stdsc = stdsc.fit_transform(d.reshape(-1,1))
    np.save(rf"Z:\200 Produced_data\MashineLearning\trainD\case{case}_{tool}",d_stdsc)
    np.save(rf"Z:\200 Produced_data\MashineLearning\trainD\case{case}_{tool}_mean",stdsc.mean_)
    np.save(rf"Z:\200 Produced_data\MashineLearning\trainD\case{case}_{tool}_std",np.sqrt(stdsc.var_))
    
    
    make_testD(case, tool, fq, AE_s_l)

In [4]:
def make_testD(case, tool, fq, AE_s_l):
    
    mean = np.load(rf"Z:\200 Produced_data\MashineLearning\trainD\case{case}_{tool}_mean.npy")
    std = np.load(rf"Z:\200 Produced_data\MashineLearning\trainD\case{case}_{tool}_std.npy")
    
    for test_tool in ["A","B","C"]:
        if test_tool == "A":
            date = "20230113+0117"
            interval = 1200
        if test_tool == "B":
            date = "20230120"
            interval = 1200
        if test_tool == "C":
            date = "20230123"
            interval = 1023
    
        d = np.array([])

        for i in tqdm(range(1,461)):
            d_n = np.load(rf"Z:\200 Produced_data\{date}\FFT\{AE_s_l}\{i}.npy")
            d_n = d_n.reshape(interval,820)
            d_n = d_n.reshape(interval,10,82)
            d_n = d_n[:,[fq]]
            d_n = d_n.reshape(interval,82*len(fq))
            d_n = d_n.ravel()
            d = np.append(d,d_n)

        test_stdsc = (d - mean) / std

        np.save(rf"Z:\200 Produced_data\MashineLearning\testD\case{case}_{tool}_{test_tool}", test_stdsc)

## case1

In [5]:
%%time
# case_info
caseNum = 1
trainTool ="A"
AE_location = "spindle"
fq = np.array([5,6])
fq_num = len(fq)
path_start = 51
path_end = 60
dict1 = dict(caseNum=caseNum, trainTool=trainTool, AE_location=AE_location, fq=[fq], fq_num=fq_num, path_start=path_start, path_end=path_end)
df1 = pd.DataFrame(data=dict1)
df1.to_csv(rf"Z:\200 Produced_data\MashineLearning\case_info.csv", mode='a', header=False)


# make trainD and testD
fq = fq-1
make_trainD(caseNum, trainTool, path_start, path_end, fq, AE_location)

100%|██████████| 460/460 [01:41<00:00,  4.53it/s]


Wall time: 5min 44s
